### get the Cohere API key from the environment variables

In [1]:
import os
from dotenv import load_dotenv

# Load the environment variables from the .env file
load_dotenv()

cohere_api_key = os.getenv("COHERE_API_KEY")

cohere_api_key

'urAN0hpVmnaPAW66RPqZQTKOSs46zd4EI93e1NNg'

### scrap website data and load it to documents

In [2]:
import bs4
from langchain_community.document_loaders import WebBaseLoader
# Load, chunk and index the contents of the blog.
loader = WebBaseLoader(
    web_paths=("https://www.mql5.com/en/blogs/post/752096", ),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("content")
        )
        # mimic the behavior of a web browser to make the request 
    ),header_template={
      'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36',
  }
)
docs = loader.load()

docs

[Document(page_content='\nHere you will find the list of the frequently asked questions and the answers to them. General Questions\xa0 How do I install expert advisors or indicators from the mql5.com market? Kindly read my\xa0installation guide. How do I update expert advisors or indicators from the mql5.com market to latest versions? The procedure is identical to the installation, but instead of the "Install" button, you should click the "Update" button (it will be available if an update is available). The free download button is not working! In order to install a free product, you need to find it in the Market and click "Download". After that, the product will immediately appear in the "Market" list of the Navigator (View => Navigator). Do I have to pay for future updates? No, updates are free. What is an activation? An activation is the act of installing a product in a Metatrader platform. Each time you do, an activation is consumed.\xa0 Does an update consume an activation? No, it 

### text processing and splitting 

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
splits

[Document(page_content='Here you will find the list of the frequently asked questions and the answers to them. General Questions\xa0 How do I install expert advisors or indicators from the mql5.com market? Kindly read my\xa0installation guide. How do I update expert advisors or indicators from the mql5.com market to latest versions? The procedure is identical to the installation, but instead of the "Install" button, you should click the "Update" button (it will be available if an update is available). The free download button is not working! In order to install a free product, you need to find it in the Market and click "Download". After that, the product will immediately appear in the "Market" list of the Navigator (View => Navigator). Do I have to pay for future updates? No, updates are free. What is an activation? An activation is the act of installing a product in a Metatrader platform. Each time you do, an activation is consumed.\xa0 Does an update consume an activation? No, it do

### create a Chroma vector store 

In [4]:
from langchain_chroma import Chroma
from langchain_community.embeddings import CohereEmbeddings

cohere_embeddings = CohereEmbeddings(model="embed-english-light-v3.0", cohere_api_key=cohere_api_key)
vectorstore = Chroma.from_documents(documents=splits, embedding=cohere_embeddings)


c:\Users\Admin\miniconda3\envs\chatbot\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `CohereEmbeddings` was deprecated in LangChain 0.0.30 and will be removed in 0.2.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import CohereEmbeddings`.
  warn_deprecated(


### retrieve and generate using the relevant snippets of the website data

In [5]:
retriever = vectorstore.as_retriever()

In [6]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate

# create prompt
system_template = """You are an AI assistant tasked with answering questions based on the given context. 
Always strive for accuracy and clarity in your responses."""

human_template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context:
{context}

Question: {question}

Your answer:"""

system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])
chat_prompt

ChatPromptTemplate(input_variables=['context', 'question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an AI assistant tasked with answering questions based on the given context. \nAlways strive for accuracy and clarity in your responses.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], template="Use the following pieces of context to answer the question at the end. \nIf you don't know the answer, just say that you don't know, don't try to make up an answer.\n\nContext:\n{context}\n\nQuestion: {question}\n\nYour answer:"))])

In [7]:
from langchain.llms import Cohere
llm = Cohere()

c:\Users\Admin\miniconda3\envs\chatbot\Lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The class `Cohere` was deprecated in LangChain 0.1.14 and will be removed in 0.2.0. An updated version of the class exists in the langchain-cohere package and should be used instead. To use it run `pip install -U langchain-cohere` and import as `from langchain_cohere import Cohere`.
  warn_deprecated(


In [8]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

### create a rag chain to retrieve data and output

In [9]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | chat_prompt
    | llm
    | StrOutputParser()
)

In [10]:
rag_chain.invoke("Do I have to pay for future updates for mt5?")

' The answer is no. Future updates for MT5 do not have any cost attached to them and they are free to download and update with the latest versions. '

In [11]:
rag_chain.invoke("what is mql5?")

' MQL5 is an integrated development environment (IDE) for developing trading strategies and technical indicators for MetaTrader 5 trading platform. It can also be used with MetaTrader 4. Developers can use MQL5 to compile, debug, and troubleshoot programs, and the MQL5 Community to share and receive trading strategies and technical indicators. '

In [12]:
rag_chain.invoke("what is ice cream?")

' Ice cream is a popular frozen dessert usually made from a mixture of milk, cream, and sweeteners with added flavorings. It is a well-known and widely consumed food product, appreciated for its rich taste, creamy texture, and cooling sensation. Ice cream is often flavored with a variety of ingredients, including chocolate, vanilla, nuts, fruits, and spices. It is typically served at a cold temperature and can be enjoyed in various forms, such as cones, cups, sundaes, shakes, and baked goods. Ice cream is celebrated and enjoyed globally, holding a special place in many cultures and serving as a popular snack and comfort food. \n\nHowever, ice cream is high in calories and sugar and contains saturated fat, so it should be enjoyed in moderation as part of a balanced lifestyle. \n\nI hope this answers your question! Let me know if you would like any suggestions on delicious flavors to try. '

### contextualizing the question

In [13]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)
contextualize_q_prompt

ChatPromptTemplate(input_variables=['chat_history', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='Given a chat history and the latest user question which might reference context in the chat history, formulate a standalone question which can be understood without the chat history. Do NOT answer the question, just reformulate it if needed and otherwise return it as is.')), MessagesPlaceholder(variable_name='chat_history'), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

### build up QA chain with chat history

In [14]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_system_prompt = """You are an assistant for question-answering tasks. \
Use the following pieces of retrieved context to answer the question. \
If you don't know the answer, just say that you don't know. \
Use three sentences maximum and keep the answer concise.\

{context}"""
qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [15]:
from langchain_core.messages import HumanMessage

chat_history = []

question = "what is mql5?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend([HumanMessage(content=question), ai_msg_1["answer"]])

second_question = "does it offer discounts?"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

 Unlike other marketplaces, mql5 does not offer discounts. However, some sellers may provide promotions or bundle deals on their products. These deals are sometimes posted on the news feed, so it's best to add sellers as friends on the platform. 


### returning sources

In [16]:
for document in ai_msg_2["context"]:
    print(document)
    print()

page_content="activation is the act of installing a product in a Metatrader platform. Each time you do, an activation is consumed.\xa0 Does an update consume an activation? No, it does not.\xa0 Can I get the .mql file (source code) of a product? No, we don't disclose our source codes for any reason. Can I run indicators and EAs from the mql5.com market in Mac OS X? Yes, but you need to virtualize Windows on your Mac. You can use Parallels Desktop or VMWare to do so. You can also use remote access to a server (for example, using Microsoft Remote Desktop), where Windows and the Metatrdader 4 or 5 platform are installed. Emulation with Wine or Crossover won't work because the mql5.com market needs Internet Explorer (IE > 8) to be installed in your operating system. Do you offer discounts? No, we don't. The mql5.com market does not implement discount codes functionality. But sometimes I have promotions for my products. You can find out about them in the news feed if you add me as a friend.